# Title : Machine Learning For Detecting Hate Speech In Social Media

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

The dataset we are using for the hate speech detection task is downloaded from Kaggle. This dataset was originally collected from Twitter and contains the following columns:

1.index

2.count

3.hate_speech

4.offensive_language

5.neither 

6.class

7.tweet 

So let’s start by importing all the necessary Python libraries and the dataset we need for this task:

In [2]:
from nltk.util import pr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))
data = pd.read_csv("labeled_data.csv")
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


# We will add a new column to this dataset as labels which will contain the values as:

1.Hate Speech 

2.Offensive Language 

3.No Hate and Offensive

In [3]:
data["labels"] = data["class"].map({0: "Hate Speech", 
                                    1: "Offensive Language", 
                                    2: "No Hate and Offensive"})
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet                 labels  
0  !!! RT @mayasolovely: As a woman you shouldn't...  No Hate and Offensive  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...     Offensive Language  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...     Offensive Language  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...     Offensive Language  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...     Offensive Language  


# Now We will only select the tweet and labels columns for the rest of the task of training a hate speech detection model:

In [4]:
data = data[["tweet", "labels"]]
print(data.head())

                                               tweet                 labels
0  !!! RT @mayasolovely: As a woman you shouldn't...  No Hate and Offensive
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...     Offensive Language
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...     Offensive Language
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...     Offensive Language
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...     Offensive Language


# Now We will create a function to clean the texts in the tweet column:

In [5]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)

# Now let’s split the dataset into training and test sets and train a machine learning model for the task of hate speech detection:

In [6]:
x = np.array(data["tweet"])
y = np.array(data["labels"])

In [7]:
cv = CountVectorizer()
X = cv.fit_transform(x) # Fit the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier()

# Now let’s test this machine learning model to see if it detects hate speech or not:

In [9]:
sample = "Let's unite and kill all the people who are protesting against the government"
data = cv.transform([sample]).toarray()
print(clf.predict(data))

['Hate Speech']


In [12]:
sample = input("Please enter your speech: ")
data = cv.transform([sample]).toarray()
print(clf.predict(data))

Please enter your speech:  she is the fucking bitch


['Offensive Language']


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the test set
y_pred = clf.predict(X_test)

# Generate classification report
class_names = ['Hate Speech', 'Offensive Language', 'No Hate and Offensive']
report = classification_report(y_test, y_pred, target_names=class_names)
print("Classification Report: \n")
print(report)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix: \n")
print(conf_matrix)

Classification Report: 

                       precision    recall  f1-score   support

          Hate Speech       0.37      0.33      0.35       465
   Offensive Language       0.82      0.81      0.82      1379
No Hate and Offensive       0.92      0.93      0.93      6335

             accuracy                           0.88      8179
            macro avg       0.70      0.69      0.70      8179
         weighted avg       0.87      0.88      0.87      8179


Confusion Matrix: 

[[ 154   29  282]
 [  37 1121  221]
 [ 227  215 5893]]


# Save the model

**Export as a Pickle file**: We use Python's pickle module to serialize the trained model object and save it to a file. Others can then deserialize the file and use the model for predictions.

In [17]:
import pickle

In [19]:
pickle.dump(clf,open("hatespeech.pkl","wb"))

In [23]:
load=pickle.load(open("hatespeech.pkl","rb"))

In [26]:
print(f"{load.score(X_train,y_train)*100:.2f}%")

99.96%
